In [12]:
from skfeature.function.information_theoretical_based import MRMR

from skfeature.function.information_theoretical_based import MRMR
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from mat4py import loadmat
import scipy.io
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def extract_data(dset):
    epoch1 = pd.read_excel(dset)
    labels_all = pd.read_excel('label_for_ut47_session0.xlsx')
    
#########################################################################################
#Cal of corr matrix
    epoch1_corrmat=epoch1.corr()
    cordf = pd.DataFrame(epoch1_corrmat)

#########################################################################################
#Cal of Mutual info score
    
    mi = epoch1.corr(method='spearman')
    mi_df = pd.DataFrame(mi)

    matrix = []

    for i in range(0, 64):
        row = []
        matrix.append(row)
        for j in range(0, 64):
            if i == j:
                row.append(1)
            elif i>j:
                row.append(cordf.iloc[i,j])
            else:
                row.append(mi_df.iloc[i,j])

    rel_df = pd.DataFrame(matrix)
    # cal of bandpowers

    import pyeeg

    def BandPower( x ):

        fs = 128
        band = [1,4,8,12,30]

        resp = pyeeg.bin_power(x,band,fs)

        return resp
    import numpy as np
    dfn=epoch1.to_numpy()
    dfn1 = dfn.T

    a=[]
    b=[]
    for i in range(len(dfn1)):
    
        fs = 200
        band = [1,4,8,13,30,100]

        resp = pyeeg.bin_power(dfn1[i],band,fs)
        a.append(resp[0])
        b.append(resp[1])
#pb = pd.DataFrame(a,columns=['Delta','Theta','Alpha','Beta','Gamma'])
    pr = pd.DataFrame(b,columns=['Ratio Delta','Ratio Theta','Ratio Alpha','Ratio Beta','Ratio Gamma'])
#prodata=pd.concat([pb,pr],axis=1)

###########################################################################################
#Cal of stats

    m =pd.DataFrame(epoch1.mean(),columns=['mean'])
    k =pd.DataFrame(epoch1.kurtosis(),columns=['kurtosis'])
    sk= pd.DataFrame(epoch1.skew(),columns=['skew'])
    st=pd.DataFrame(epoch1.std(),columns=['std'])
    all_stats = pd.concat([m,k,sk,st],axis=1)
#stats = pd.DataFrame([m,k,sk,st],columns=['mean','kurtosis','skew','std'])

###########################################################################################
#final test/train dataset

    final_df = pd.concat([rel_df,all_stats,pr],axis=1)
 # MRMR   
    
    #print(final_df.shape)
    
    return final_df

In [3]:
train_data = []

DATADIR = 'D:/subjFiles/UT047/behavioral/FR1/epochs eeg for session 0  uta47/data'
CATEGORIES=['Recalled','Not_recalled']

def create_traindata():
    
    for category in CATEGORIES:
        #print(category)
        path = os.path.join(DATADIR, category)
        #print(path)
        class_lab = CATEGORIES.index(category)
        
        
        for ex in os.listdir(path):
            try:
                
                ext_data = extract_data(ex)
                datarry = np.array(ext_data)
                
                
                
                
                train_data.append([datarry,class_lab])
                
            except Exception as e:
                pass
            

            
create_traindata()

In [4]:
import  random
#random.shuffle(train_data)
feat_all=[]
label_all=[]
for features,label in train_data:
    feat_all.append(features)
    label_all.append(label)
X = np.array(feat_all).reshape(300,4672)
y = np.array(label_all)
y_df = pd.DataFrame(label_all)
X.shape

(300, 4672)

In [5]:
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
scale = MinMaxScaler()
new_df = scale.fit_transform(X)
#sns_plot=sns.heatmap(new_df,xticklabels=2,yticklabels=False,square=True)
#sns_plot.figure.savefig('testfig.png')



In [6]:
X = pd.DataFrame(new_df)
y = pd.DataFrame(y)
X.shape

(300, 4672)

In [7]:
X=X.values
X=X.astype(float)
y1= y.values
y2=y1[:,0]
n_samples, n_features = X.shape 
num_fea = 100
correct = 0
# obtain the index of each feature on the training set
idx,_,_ = MRMR.mrmr(X, y2, n_selected_features=num_fea)
 
    # obtain the dataset on the selected features
features = X[:, idx[0:num_fea]]

In [8]:
X1 = np.array(features)
X1.shape

(300, 100)

In [130]:
from sklearn.model_selection import train_test_split

#X_train,X_test,y_train,y_test=train_test_split(X1,y2,test_size=0.10,stratify = y2)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X1,y2,test_size=0.20,stratify=y2,random_state=0)

In [131]:
X_train.shape

(240, 100)

In [208]:
#model 1 logistic regression
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(max_iter=1000, solver='lbfgs',multi_class = 'auto')
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

#use print only if train_test_split is used
print(classification_report(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))   

              precision    recall  f1-score   support

           0       0.67      0.09      0.15        23
           1       0.63      0.97      0.77        37

    accuracy                           0.63        60
   macro avg       0.65      0.53      0.46        60
weighted avg       0.65      0.63      0.53        60

[[ 2 21]
 [ 1 36]]


In [243]:
#Model2 Random forest with entropy
from sklearn.ensemble import RandomForestClassifier


model2=RandomForestClassifier(max_depth=7,n_estimators=10,min_samples_split=10,criterion='entropy')
model2.fit(X_train,y_train)
y_predrf=model2.predict(X_test)


#use print only if train_test_split is used
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predrf))
print(classification_report(y_test,y_predrf))
print(confusion_matrix(y_test,y_predrf))

0.6166666666666667
              precision    recall  f1-score   support

           0       0.50      0.35      0.41        23
           1       0.66      0.78      0.72        37

    accuracy                           0.62        60
   macro avg       0.58      0.57      0.56        60
weighted avg       0.60      0.62      0.60        60

[[ 8 15]
 [ 8 29]]


In [206]:
import xgboost as xgs
model3 = xgs.XGBClassifier(max_depth=11,n_estimators=300)
model3.fit(X_train,y_train)
ypred3 = model3.predict(X_test)

#use print only if train_test_split is used
#from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,ypred3))
print(classification_report(y_test,ypred3))
print(accuracy_score(y_test,ypred3))

[[10 13]
 [11 26]]
              precision    recall  f1-score   support

           0       0.48      0.43      0.45        23
           1       0.67      0.70      0.68        37

    accuracy                           0.60        60
   macro avg       0.57      0.57      0.57        60
weighted avg       0.59      0.60      0.60        60

0.6


In [230]:
#model 4 SVM
from sklearn.svm import SVC
model4 = SVC(C=500,gamma=0.01,probability=True,)
model4.fit(X_train,y_train)
ypred4 = model4.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
#use print only if train_test_split is used
print(accuracy_score(y_test,ypred4))
print(confusion_matrix(y_test,ypred4))
print(classification_report(y_test,ypred4))

0.7333333333333333
[[11 12]
 [ 4 33]]
              precision    recall  f1-score   support

           0       0.73      0.48      0.58        23
           1       0.73      0.89      0.80        37

    accuracy                           0.73        60
   macro avg       0.73      0.69      0.69        60
weighted avg       0.73      0.73      0.72        60



In [165]:
#model 5 random forst with information gain or gini
from sklearn.ensemble import RandomForestClassifier
model6=RandomForestClassifier(class_weight = 'balanced',max_depth=1000,n_estimators=3,min_samples_split=10,criterion='gini')
model6.fit(X_train,y_train)
y_predrf6=model6.predict(X_test)


#use print only if train_test_split is used
print(classification_report(y_test,y_predrf6))
print(confusion_matrix(y_test,y_predrf6))

              precision    recall  f1-score   support

           0       0.22      0.22      0.22        23
           1       0.51      0.51      0.51        37

    accuracy                           0.40        60
   macro avg       0.37      0.37      0.37        60
weighted avg       0.40      0.40      0.40        60

[[ 5 18]
 [18 19]]


In [212]:

from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
estimators=[('RF1', model2),('svm',model4),('xgb',model3)]
#create our voting classifier, inputting our models
model5 = VotingClassifier(estimators, voting='soft')
#fit model to training data
model5.fit(X_train,y_train)
#test our model on the test data
ypred_en=model5.predict(X_test)

#use print only if train_test_split is used
print(model5.score(X_test, y_test))

print(confusion_matrix(y_test,ypred_en))
print(classification_report(y_test,ypred_en))

0.6
[[ 5 18]
 [ 6 31]]
              precision    recall  f1-score   support

           0       0.45      0.22      0.29        23
           1       0.63      0.84      0.72        37

    accuracy                           0.60        60
   macro avg       0.54      0.53      0.51        60
weighted avg       0.56      0.60      0.56        60



In [225]:
# finding best params for svm

from sklearn.model_selection import GridSearchCV
param_grid = {'C' : [ 1, 10, 100,200,300,400,600,500,700,800, 1000 ], 'gamma': [1,0.1,0.0001]}

In [240]:
grid = GridSearchCV(SVC(probability=True),param_grid,verbose=2)
grid.fit(X_train,y_train)

C:\Users\AB Duke\Anaconda4\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 33 candidates, totalling 99 fits
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=   0.0s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=   0.0s
[CV] C=1, gamma=1 ....................................................
[CV] ..................................... C=1, gamma=1, total=   0.0s
[CV] C=1, gamma=0.1 ..................................................
[CV] ................................... C=1, gamma=0.1, total=   0.0s
[CV] C=1, gamma=0.1 ..................................................
[CV] ................................... C=1, gamma=0.1, total=   0.0s
[CV] C=1, gamma=0.1 ..................................................
[CV] ................................... C=1, gamma=0.1, total=   0.0s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ...........

[CV] ................................. C=600, gamma=0.1, total=   0.1s
[CV] C=600, gamma=0.1 ................................................
[CV] ................................. C=600, gamma=0.1, total=   0.0s
[CV] C=600, gamma=0.1 ................................................
[CV] ................................. C=600, gamma=0.1, total=   0.0s
[CV] C=600, gamma=0.0001 .............................................
[CV] .............................. C=600, gamma=0.0001, total=   0.0s
[CV] C=600, gamma=0.0001 .............................................
[CV] .............................. C=600, gamma=0.0001, total=   0.0s
[CV] C=600, gamma=0.0001 .............................................
[CV] .............................. C=600, gamma=0.0001, total=   0.0s
[CV] C=500, gamma=1 ..................................................
[CV] ................................... C=500, gamma=1, total=   0.0s
[CV] C=500, gamma=1 ..................................................
[CV] .

[Parallel(n_jobs=1)]: Done  99 out of  99 | elapsed:    2.8s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 10, 100, 200, 300, 400, 600, 500, 700, 800,
                               1000],
                         'gamma': [1, 0.1, 0.0001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [238]:
grid_pridiction = grid.predict(X_test)

In [239]:
grid.best_params_

{'C': 1, 'gamma': 0.1}

In [234]:
from sklearn.neighbors import KNeighborsClassifier
modelk = KNeighborsClassifier(n_neighbors=5)
modelk.fit(X_train,y_train)
ypredk = modelk.predict(X_test)
print(modelk.score(X_test, y_test))
print(confusion_matrix(y_test,ypredk))
print(classification_report(y_test,ypredk))

0.6666666666666666
[[10 13]
 [ 7 30]]
              precision    recall  f1-score   support

           0       0.59      0.43      0.50        23
           1       0.70      0.81      0.75        37

    accuracy                           0.67        60
   macro avg       0.64      0.62      0.62        60
weighted avg       0.66      0.67      0.65        60



In [179]:
from sklearn.metrics import  recall_score

In [180]:
recall_score(X_test, y_test)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets